# Merge-based spMspM

This is an example of A-stationary/row-major (Gusstavson) spM-spM, using a merge based accelerator.

First, include some libraries

In [ ]:
%run ../prelude.py --style=tree --animation=movie

In [ ]:
M = 6
K = 6
N = 6
seed = 20

a = Tensor.fromRandom(["M", "K"], [M, K], [0.7, 0.8], 5, seed=seed)
a.setColor("blue").setName("A")
displayTensor(a)

b = Tensor.fromRandom(["K", "N"], [K, N], [0.7, 0.65], 5, seed=seed)
b.setColor("green").setName("B")
displayTensor(b)



## A-stationary/row-major spMspM

Basic algorithm

In [ ]:
z = Tensor(rank_ids=["M", "N"], shape=[M,N])
z.setName("Z")

a_m = a.getRoot()
b_k = b.getRoot()
z_m = z.getRoot()

canvas = createCanvas(a, b, z)

for m, (z_n, a_k) in z_m << a_m:
    for k, (a_val, b_n) in a_k & b_k:
        for n, (z_ref, b_val) in z_n << b_n:
            z_ref += a_val * b_val
            addFrame(canvas, (m, k), (k, n), (m, n))

displayTensor(z)
displayCanvas(canvas)

## Merge-based - with infinite merger

Using an append to add swapped ranks into "ba" tensor

In [ ]:
z = Tensor(rank_ids=["M", "N"], shape=[M,N])
z.setName("Z")

ba = Tensor(rank_ids=["M", "N", "K"])
ba.setColor("purple")
ba.setName("BA")

b_swapped = b.swapRanks()

a_m = a.getRoot()
b_k = b.getRoot()
ba_m = ba.getRoot()
z_m = z.getRoot()

canvas = createCanvas(a, b, b_swapped, ba, z)

for m, (z_n, a_k) in z_m << a_m: # M-dimension parallelism

    ba_k1 = b_k & a_k

    ba_n = ba_k1.swapRanks()  # implemented with a merger and pipelined with next loop
    ba_m.append(m, ba_n)
    
    for n, (z_ref, ba_k) in z_n << ba_n:
        for k, (b_val, a_val) in ba_k:
            z_ref += a_val * b_val
            addFrame(canvas, (m, k), (k, n), (n, k), (m, n, k), (m, n))

displayTensor(z)
displayCanvas(canvas)

Basically the same code as above but creating the intersection fiber ab_k rather than ba_k. In this case, the second element of the intersection tuple is a Fiber and the swapRanks() was failing because of a problem in flattenRanks(). This is a test that those problems have been fixed...

In [ ]:
z = Tensor(rank_ids=["M", "N"], shape=[M,N])
z.setName("Z")

ab = Tensor(rank_ids=["M", "N", "K"])
ab.setColor("purple")
ab.setName("AB")

b_swapped = b.swapRanks()

a_m = a.getRoot()
b_k = b.getRoot()
ab_m = ab.getRoot()
z_m = z.getRoot()

canvas = createCanvas(a, b, b_swapped, ab, z)

for m, (z_n, a_k) in z_m << a_m: # M-dimension parallelism

    ab_k1 = a_k & b_k

    ab_n = ab_k1.swapRanks()  # implemented with a merger and pipelined with next loop
    ab_m.append(m, ab_n)
    
    for n, (z_ref, ab_k) in z_n << ab_n:
        for k, (a_val, b_val) in ab_k:
            z_ref += a_val * b_val
            addFrame(canvas, (m, k), (k, n), (n, k), (m, n, k), (m, n))

displayTensor(z)
displayCanvas(canvas)